# SimNet Training for Balancing Robot

This notebook trains a SimNet model using both physics simulation data and real-world data.

In [1]:
import sys
sys.path.append('..')

import torch
import numpy as np
from pathlib import Path
import json

from src.balancing_robot.models import SimNet
from src.balancing_robot.environment import BalancerEnv
from src.balancing_robot.training import SimNetTrainer
from src.balancing_robot.visualization import plot_predictions_comparison

## Initialize Models and Environment

In [10]:
# Set random seeds
torch.manual_seed(42)
np.random.seed(42)

# Create environment for physics data
env = BalancerEnv()

# Initialize SimNet
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.shape[0]
simnet = SimNet(
    state_dim=state_dim,
    action_dim=action_dim,
    hidden_dims=(128, 128),
    learning_rate=1e-3
)

# Initialize trainer
trainer = SimNetTrainer(
    env=env,
    config_path=Path('../configs/simnet_config.yaml'),
)

## Collect and Process Physics Data

In [13]:
# Collect physics simulation data
num_samples = 100000
train_data, val_data = trainer.collect_physics_data()

print(f"Collected {len(train_data['states'])} training samples")
print(f"Collected {len(val_data['states'])} validation samples")

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

## Train on Physics Data

In [ ]:
# Create log directory
log_dir = Path('logs/simnet_training')
log_dir.mkdir(parents=True, exist_ok=True)

# Train on physics data
physics_history = trainer.train(
    train_data=train_data,
    val_data=val_data,
    num_epochs=50,
    batch_size=512,
    log_dir=log_dir / 'physics'
)

## Load and Process Real-World Data

In [ ]:
# Load real-world log data
log_file = Path('data/robot_logs.json')
with open(log_file, 'r') as f:
    log_data = json.load(f)

# Process real data
real_train_data, real_val_data = trainer.process_real_data(log_data['episodes'])

print(f"Processed {len(real_train_data['states'])} real training samples")
print(f"Processed {len(real_val_data['states'])} real validation samples")

## Fine-tune on Real Data

In [ ]:
# Fine-tune on real data
real_history = trainer.train(
    train_data=real_train_data,
    val_data=real_val_data,
    num_epochs=20,
    batch_size=128,
    log_dir=log_dir / 'real'
)

## Analyze Predictions

In [ ]:
# Compare physics and SimNet predictions
test_states = val_data['states'][:1000]
test_actions = val_data['actions'][:1000]

physics_preds = np.array([env.physics.get_acceleration(state, action) 
                         for state, action in zip(test_states, test_actions)])

simnet_preds = np.array([simnet.get_accelerations(state, action)
                         for state, action in zip(test_states, test_actions)])

# Plot comparison
fig = plot_predictions_comparison(
    physics_preds=physics_preds,
    simnet_preds=simnet_preds,
    save_path=log_dir / 'prediction_comparison.png'
)
plt.show()

## Save Final Model

In [ ]:
# Save trained SimNet
torch.save({
    'state_dict': simnet.state_dict(),
    'physics_history': physics_history,
    'real_history': real_history,
    'metadata': {
        'state_dim': state_dim,
        'action_dim': action_dim,
        'hidden_dims': (128, 128)
    }
}, log_dir / 'simnet_final.pt')